In [123]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
! pip install sentencepiece

import sentencepiece as spm
import unicodedata
import re
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import os
import io
import time

!pip install japanize-matplotlib
import japanize_matplotlib
tf.enable_eager_execution()

In [0]:
# load txt file
def load_def(path):
    # open a txt file as read only
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    return lines

In [0]:
# create each languages list
def create_lang_list(num_example):
    # load txt file
    lines = load_def("/content/drive/My Drive/Colab Notebooks/raw.txt")

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_example]]

    return zip(*word_pairs)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# translate English to Japanese

In [0]:
sp = spm.SentencePieceProcessor()
# train SentencePiece using english text 
spm.SentencePieceTrainer.Train('--input=english.txt --model_prefix=m --vocab_size=8000')
# convert unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFC', s)
                    if unicodedata.category(c) != 'Mn')

# preprocess
def preprocess_sentence(w):
    # check japanese lang
    # p = re.compile('[\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F]+')
    # if p.search(w):
        # Morphological analysis for japanese lang
        # english sentencepiece

    w = unicode_to_ascii(w.lower().strip())
    # create a space between word and the punctuation
    w = re.sub(r"([?!¿.,。])", r" \1 ", w)
    # replacing everything with space except(a-z, A-Z, ".",  "?",  "!",  ",", "-", "ー", , "。", "Kanji", "Katakana", "Hiragana")
    w = re.sub(r"[^a-zA-Z\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F?.!,。¿\-/ {1,}/]+",  " ", w)
    w = w.rstrip().strip()
    
    # add a start and end  token to the sentence
    # model know when to start and end
    w = "<start> " + w + " <end>"
    return w

In [66]:
# check word
en_sentence =u"May I borrow this book?"
ja_sentence = u"プールに行きたい。でも今日は雨."
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(ja_sentence))

<start> may i borrow this book ? <end>
<start> プールに行きたい 。 でも今日は雨 . <end>


In [67]:
en, ja = create_lang_list(100)
print(len(en))
print(ja[:10])

100
('<start> あなたは戻ったのね ハロルド ? <end>', '<start> 俺の相手は シャークだ 。 <end>', '<start> 引き換えだ ある事とある物の <end>', '<start> もういいよ ごちそうさま ううん <end>', '<start> もう会社には来ないでくれ 電話もするな <end>', '<start> きれいだ 。 <end>', '<start> 連れて行け 殺しそうだ わかったか ? <end>', '<start> 殺したのか ! <end>', '<start> わぁ  !  いつも すみません 。  いいのよ  。 <end>', '<start> カンパニーの元社員が <end>')


In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
    # vectorize a text corpus
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=' ')
    # updates internal vocabulary based on a list of texts
    # e.g. "[this place is good ]"→{this:1, place:2, is:3, good:4} "
    lang_tokenizer.fit_on_texts(lang)
    # Transforms each text in texts to a sequence of integers.
    tensor = lang_tokenizer.texts_to_sequences(lang)
    # transform a list of num sample into a 2D Numpy array of shape 
    # Fixed length because length of sequence of integers are different
    # return (len(sequences), maxlen)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                          padding='post')
    return tensor, lang_tokenizer

In [70]:
# example
tokenize(['this place is good', "こんにちは 今日は いい天気 今日", "today is so cold"])

(array([[ 2,  3,  1,  4],
        [ 5,  6,  7,  8],
        [ 9,  1, 10, 11]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x7f608a58b630>)

In [0]:
# create a clean input, output pairs
def load_dataset(num_example):
    input_lang, target_lang= create_lang_list(num_example)
    sp.Load('/content/drive/My Drive/wiki-ja.model')
    target_lang = ["".join(sp.encode_as_pieces(i)).replace("▁", " ") for i in target_lang]
    print(target_lang[:10])
    input_tensor, input_lang_tokenize = tokenize(input_lang)
    target_tensor, target_lang_tokenize = tokenize(target_lang)

    return input_tensor, target_tensor, input_lang_tokenize, target_lang_tokenize

In [84]:
# limit datasize for test
num_example = 10000
# get data
input_tensor, target_tensor, input_lang, target_lang = load_dataset(num_example)
# Calculate max_length of the target tensors
max_length_target, max_length_input = max_length(target_tensor), max_length(input_tensor)

[' <start> あなたは戻ったのね ハロルド ? <end>', ' <start> 俺の相手は シャークだ 。 <end>', ' <start> 引き換えだ ある事とある物の <end>', ' <start> もういいよ ごちそうさま ううん <end>', ' <start> もう会社には来ないでくれ 電話もするな <end>', ' <start> きれいだ 。 <end>', ' <start> 連れて行け 殺しそうだ わかったか ? <end>', ' <start> 殺したのか ! <end>', ' <start> わぁ ! いつも すみません 。 いいのよ 。 <end>', ' <start> カンパニーの元社員が <end>']


In [85]:
# create trainnig set and validation set
input_tensor_train, input_tensor_val, \
    target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=42)

# show length
print(len(input_tensor_train), len(input_tensor_val), len(target_tensor_train), len(target_tensor_val))

8000 2000 8000 2000


In [0]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            # Index number assigned to each word
            print("%d----->%s" % (t, lang.index_word[t]))

In [87]:
print("input lang: index to word mapping")
convert(input_lang, input_tensor_train[3])
print("output lang: index to word mapping")
convert(target_lang, target_tensor_train[3])

input lang: index to word mapping
1-----><start>
21----->this
6550----->script
260----->wasn
18----->t
1015----->written
110----->by
6551----->takakura
6552----->yuuji
3----->.
2-----><end>
output lang: index to word mapping
1-----><start>
9703----->この脚本は
9704----->高倉
9705----->雄二が
9706----->書いたものでは
719----->ありません
5----->。
2-----><end>


In [90]:
# create a dataset
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 128
steps_per_epoch = len(input_tensor_train)
embedding_dim = 100
units = 512
vocab_inp_size = len(input_lang.word_index) + 1
print('Total unique words in the input: %s' % len(input_lang.word_index))
vocab_tar_size = len(target_lang.word_index) + 1
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
print(dataset)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

Total unique words in the input: 8560
<DatasetV1Adapter shapes: ((55,), (16,)), types: (tf.int32, tf.int32)>
<DatasetV1Adapter shapes: ((128, 55), (128, 16)), types: (tf.int32, tf.int32)>


In [91]:
example_input_batch, example_target_batch =  next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([Dimension(128), Dimension(55)]),
 TensorShape([Dimension(128), Dimension(16)]))

# encoder and decoder model

In [0]:
def load_glove():
    embeddings_dictionary = {}
    glove_file = open("/content/drive/My Drive/glove.6B.100d.txt", encoding="utf-8")
    
    for line in glove_file:
        records = line.split()
        word = records[0]
        vector_dimensions = asarray(records[1:], dtype="float32")
        # correspond word and vector
        embeddings_dictionary[word] = vector_dimensions
    glove_file.close()
    
    num_words = min(num_example, vocab_inp_size)
    embedding_matrix = zeros((num_words, embedding_dim))
    for word, index in input_lang.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embeddings_dictionary, embedding_matrix

In [0]:
embeddings_dictionary, embedding_matrix = load_glove()

In [96]:
# check vector "hello" → both of vector are almost same
print(embeddings_dictionary["today"])
print(embedding_matrix[397])

[-0.19939    0.37846    0.52093    0.28347   -0.1898    -0.20947
 -0.23286    0.14185   -0.034916  -0.36006   -0.0046717 -0.25207
  0.31364   -0.34879    0.032268  -0.45078    0.011292   0.090903
 -0.62866    0.0079579  0.28065    0.34158   -0.25593    0.11521
  0.10571   -0.45827    0.32193   -0.29186    0.11443    0.17972
 -0.31633    0.40085   -0.24405   -0.050205   0.16485    0.5001
  0.11756    0.043875  -0.060235  -0.66571   -0.40628   -0.21691
  0.068156  -0.38058   -0.4512    -0.26966    0.45961   -0.23446
 -0.26416   -1.1617     0.18417   -0.53088    0.54179    0.78295
 -0.54864   -2.0634    -0.63427    0.095073   1.9649     0.47031
 -0.54401    0.73015   -0.34352   -0.43033    0.27555    0.025185
  0.35053    0.46295    0.40837   -0.011836   0.34553   -0.17297
 -0.13765   -0.23182    0.38986    0.031124  -0.12248    0.25285
 -1.0669    -0.19626    0.42168   -0.077544  -0.37391   -0.032836
 -0.94477   -0.10812   -0.41055   -0.52736   -0.038137  -0.03078
  0.65461   -0.28335   

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size, embedding_matrix):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim, 
                                                   embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix))
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                                            return_sequences=True,
                                                            return_state=True,
                                                            recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state
        
    def initialize_hidden_state(self):
            return tf.zeros((self.batch_size, self.enc_units))

In [192]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, embedding_matrix)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (128, 55, 512)
Encoder Hidden state shape: (batch size, units) (128, 512)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, nuits):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, query, values):
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)
        
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
                self.W1(values) +  self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

In [193]:
attention_layer = BahdanauAttention(10)
attention_result , attention_weights = attention_layer(sample_hidden, sample_output)

print("attention result shape: (batch size, units) {}".format(attention_result.shape))
print("attention weight shape:(batch size, sequence_length, 1) {}".format(attention_weights.shape))

attention result shape: (batch size, units) (128, 512)
attention weight shape:(batch size, sequence_length, 1) (128, 55, 1)


In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                                            return_sequences=True,
                                                            return_state=True,
                                                            recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        self.attention = BahdanauAttention(self.dec_units)
    
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state =  self.gru(x)
                
        # output shape == (batch_size * 1, hidden_size
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output  shape == (batch_size, vocab) 
        x = self.fc(output)
        
        return  x, state, attention_weights

In [195]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _,  _ = decoder(tf.random.uniform((128, 1)),
                                                                        sample_hidden, sample_output)
print("Decoder output  shape:(batch_size, vocab size) {}".format(sample_decoder_output.shape))

Decoder output  shape:(batch_size, vocab size) (128, 15407)


# Define the optimizer and the loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# Checkpoints

In [0]:
# checkpoint_dir = '/training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
# checkpoint = tf.train.Checkpoint(optimizer=optimizer,
#                                                         encoder=encoder,
#                                                         decoder=decoder)

# train

In [0]:
@tf.function
def train_step(inp, target, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
         
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang.word_index['<start>']] * BATCH_SIZE, 1)
        
        # teacher forcing - feeding the target as a next input
        for t in range(1, target.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            
            loss += loss_function(target[:, t], predictions)
            
            # using teacher forcing
            dec_input  = tf.expand_dims(target[:, t], 1)
        
        batch_loss = (loss / int(target.shape[1]))
        variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        
        return batch_loss

In [0]:
EPOCHS = 20
for epochs in range(EPOCHS):
    start = time.time()
    
    enc_hidden = encoder.initialize_hidden_state()
    total_loss =  0
     
    for (batch, (inp, target)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, target, enc_hidden)
        total_loss += batch_loss
        
#         if batch % 100 == 0:
#             print("epoch {} batch {} loss  {: .4f}".format(epochs + 1,
#                                                                                           batch,
#                                                                                           batch_loss.numpy()))
#         # save checkpoint every 2 epochs
#         if (epochs + 1) % 2 == 0:
#             checkpoint.save(file_prefix = checkpoint_prefix)
            
        # print("epoch {} loss {: .4f}".format(epochs + 1,
        #                                                              total_loss / steps_per_epoch))
        # print("time taken for 1 epoch {} sec\n".format(time.time() - start))

In [0]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_target, max_length_input))

    sentence = preprocess_sentence(sentence)

    inputs = [input_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_input,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang.word_index['<start>']], 0)

    for t in range(max_length_target):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_lang.index_word[predicted_id] + ' '

        if target_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [0]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [0]:
translate(u'what do you want?')